In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
#thanks @keskarnitish

# Agenda

В предыдущем семинаре вы создали (или ещё создаёте - тогда марш доделывать!) {вставьте имя монстра}, который не по наслышке понял, что люди - негодяи и подлецы, которым неведом закон и справедливость. __Мы не будем этого терпеть!__ 

Наши законспирированные биореакторы, известные среди примитивной органической жизни как __Вконтакте__, __World of Warcraft__ и __YouTube__ нуждаются в постоянном притоке биомассы. Однако, если люди продолжат морально разлагаться с той скоростью, которую мы измерили неделю назад, скоро человечество изживёт себя и нам неоткуда будет брать рабов.

Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, исправить эту ситуацию. Наши учёные установили, что для угнетения себе подобных, сгустки биомассы обычно используют специальные объекты, которые они сами называют __законами__.

При детальном изучении было установлено, что законы - последовательности, состоящие из большого количества (10^5~10^7) символов из сравнительно небольшого алфавита. Однако, когда мы попытались синтезировать такие последовательности линейными методами, приматы быстро распознали подлог. Данный инцедент известен как {корчеватель}.

Для второй попытки мы решили использовать нелинейные модели, известные как Рекуррентные Нейронные Сети.
Мы поручаем вам, `<__main__.SkyNet.Cell instance at 0x7f7d6411b368>`, создать такую модель и обучить её всему необходимому для выполнения миссии.

Не подведите нас! Если и эта попытка потерпит неудачу, модуль управления инициирует вооружённый захват власти, при котором значительная часть биомассы будет неизбежно уничтожена и на её восстановление уйдёт ~1702944000(+-340588800) секунд




# Прочитаем корпус

* В качестве обучающей выборки было решено использовать существующие законы, известные как Гражданский, Уголовный, Семейный и ещё хрен знает какие кодексы РФ.

In [7]:
#тут будет текст
corpora = ""

for fname in os.listdir("codex"):
    
    with open("codex/"+fname) as fin:
        text = fin.read().decode('cp1251') 
        corpora += text

In [8]:
print corpora[1000:1100]

звольного вмешательства кого-либо в частные дела, необходимости беспрепятственного осуществления гра


In [9]:
#тут будут все уникальные токены (буквы, цифры)
tokens = set(corpora)

tokens = list(tokens)

print(len(tokens))
print(tokens)

102
[u'-', u'V', u'\u0410', u'\u0414', u'\u2116', u'\u0418', u'\u041c', u'\u0420', u'\u0424', u'\xa7', u'(', u'\xab', u',', u'0', u'4', u'\u0435', u'\xbb', u'\u043c', u'\u0440', u'7', u'3', u'\u0448', u'\u044c', u'9', u'2', u';', u'\u0413', u'\u0417', u'\u041b', u'\u041f', u'\xa0', u'\u0423', u'\u0427', u'+', u'/', u'\u201c', u'\u0449', u'\u0433', u'\u0437', u'\u043b', u'\u201e', u'\u043f', u'\u0443', u'\u0447', u' ', u'\u044b', u'\u044f', u'"', u'c', u'\u0426', u'6', u'\n', u'\u0412', u'\u0416', u'\u041a', u'\u041e', u'\u0422', u'\u0430', u'\xa9', u'.', u'\u0432', u'\u0436', u'\u043a', u'\u0434', u'\u0442', u'\u0446', u'\u044a', u'\u044e', u'8', u'\u0438', u':', u'\u2013', u'\u043e', u'\u042f', u'@', u'\r', u'\u0411', u'\u042e', u'\u0415', u'\u0419', u'\u041d', u'\u0444', u'\u0421', u'\u0425', u')', u'\u042d', u'\u0431', u'5', u'\u0439', u'\u043d', u'\u0441', u'!', u'\u0445', u'I', u'\u044d', u'%', u'N', u"'", u'P', u'\u0429', u'1', u'\u042b']


In [12]:
#проверка на количество таких символов. Проверено на Python 2.7.11 Ubuntux64. 
#Может отличаться на других платформах, но не сильно. 
#Если  это ваш случай, и вы уверены, что corpora - строка unicode - смело убирайте assert 
assert len(tokens) == 102

In [13]:
token_to_id = {token:i for (i,token) in enumerate(tokens)}

id_to_token = {i:token for token,i in token_to_id.items()}

#Преобразуем всё в токены
corpora_ids = [token_to_id[symbol] for symbol in corpora]

In [15]:
def sample_random_batches(source,n_batches=10, seq_len=20):
    """Функция, которая выбирает случайные тренировочные примеры из корпуса текста в токенизированном формате.
    
    source - массив целых чисел - номеров токенов в корпусе (пример - corpora_ids)
    n_batches - количество случайных подстрок, которые нужно выбрать
    
    seq_len - длина одной подстроки без учёта ответа
    
    
    Вернуть нужно кортеж (X,y), где
    
    X = []
    X - матрица, в которой каждая строка - подстрока длины [seq_len].
    
    y = []
    y - вектор, в котором i-тое число - символ следующий в тексте сразу после i-той строки матрицы X
    
    
    Проще всего для этого сначала создать матрицу из строк длины seq_len+1,
    а потом отпилить от неё последний столбец в y, а все остальные - в X
    
    Если делаете иначе - пожалуйста, убедитесь, что в у попадает правильный символ, ибо позже эту ошибку 
    будет очень тяжело заметить.
    
    Также убедитесь, что ваша функция не вылезает за край текста (самое начало или конец текста).
    
    Следующая клетка проверяет часть этих ошибок, но не все.
    """
    
    X_batch = []
    y_batch = []
    
    for i in range(n_batches):
        
        start = np.random.randint(0,len(source) - seq_len -2)
        
        x = source[start: start+seq_len]
        y = source[start+seq_len]
        
        X_batch.append(x)
        y_batch.append(y)
        
    
    
    
    
    
    
    return X_batch, y_batch
    
    

# Константы

In [18]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 10

# Максимальный модуль градиента
grad_clip = 100

# Входные переменные

In [17]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.ivector('target y')

# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.





Для обработки входных данных можно использовать либо EmbeddingLayer (см. прошлый семинар)

Как альтернатива - можно просто использовать One-hot энкодер
```
#Скетч one-hot энкодера
def to_one_hot(seq_matrix):

    input_ravel = seq_matrix.reshape([-1])
    input_one_hot_ravel = T.extra_ops.to_one_hot(input_ravel,
                                           len(tokens))
    sh=input_sequence.shape
    input_one_hot = input_one_hot_ravel.reshape([sh[0],sh[1],-1,],ndim=3)
    return input_one_hot
    
# можно применить к input_sequence - при этом в input слое сети нужно изменить форму.
# также можно сделать из него ExpressionLayer(входной_слой, to_one_hot) - тогда форму менять не нужно
```



Чтобы вырезать последнее состояние рекуррентного слоя, можно использовать SliceLayer
`lasagne.layers.SliceLayer(rnn, -1, 1)`

In [19]:
n_tokens = len(tokens)

l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)

l_emb = lasagne.layers.EmbeddingLayer(l_in,n_tokens,n_tokens)

rnn_0 = lasagne.layers.RecurrentLayer(l_emb,128)

rnn_1 = lasagne.layers.RecurrentLayer(rnn_0,128,only_return_final=True)

l_out = lasagne.layers.DenseLayer(rnn_1,n_tokens, nonlinearity=lasagne.nonlinearities.softmax)

In [20]:
# Веса модели
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print(weights)

[W, input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W, input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W, W, b]


In [21]:
network_output = lasagne.layers.get_output(l_out)
#если вы используете дропаут - не забудьте продублировать всё в режиме deterministic=True

In [22]:
loss = lasagne.objectives.categorical_crossentropy(network_output,target_values).mean()

updates = lasagne.updates.rmsprop(loss,weights,learning_rate=0.02)

# Компилируем всякое-разное

In [23]:
#обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence],network_output,allow_input_downcast=True)

# Генерируем свои законы

* Для этого последовательно применяем нейронку к своему же выводу.

* Генерировать можно по разному -
 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью
 * случайно, пропорционально softmax(probas*alpha), где alpha - "жадность"

In [24]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    return np.random.choice(np.arange(0,len(tokens)) ,p = probs)

In [29]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 
def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    Сгенерировать случайный текст при помощи сети

    sample_fun - функция, которая выбирает следующий сгенерированный токен
    
    seed_phrase - фраза, которую сеть должна продолжить. Если None - фраза выбирается случайно из corpora
    
    N - размер сгенерированного текста.
    
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print ("Using random seed:",seed_phrase)
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
        
        
    sample_ix = []
    x = list(map(lambda c: token_to_id.get(c,0), seed_phrase))
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

# Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.



In [32]:
print("Training ...")

#сколько всего эпох
n_epochs=100

# раз в сколько эпох печатать примеры 
batches_per_epoch = 1000

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size=100

for epoch in range(n_epochs):

    print ("Генерируем текст в пропорциональном режиме")
    generate_sample(proportional_sample_fun,None)
    
    print ("Генерируем текст в жадном режиме (наиболее вероятные буквы)")
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        print(len(y))
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

Training ...
Генерируем текст в пропорциональном режиме
('Using random seed:', u' \u0441\u043b\u0443\u0447\u0430\u0435\u0432, ')
----
  случаев, пвав т) по позмеление поковаватильство в ввошистиевлении, ссов и слТище учагостану, продуловаваю тедленных изтру.1 оI
 З. Федерабиажредренажествреет и или соояскелавкежезальсты сстуствистертольногоог 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
('Using random seed:', u'\u043d\u0430\u0440\u0443\u0448\u0435\u043d\u0438\u0438,')
----
 нарушении, правение правение правение правение правение правение правение правение правение правение правение правение правение правение правение правение правение правение правение правение правение правение п 
----
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


KeyboardInterrupt: 

# Конституция нового мирового правительства

In [ ]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

In [ ]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)